# Projeto 1 - Ciência dos Dados

Nome: Enzo Cardoso de Almeida Santos Neto

- Limpei o kernell e dei clear output após a coleta da base

___
# Obtenção de tweets

## Preparando o ambiente no jupyter:

In [ ]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [ ]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [ ]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapa 1 - Construção da base de dados:

### Escolha de um produto e coleta das mensagens


In [ ]:
#Produto escolhido:
produto = 'Valorant'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [ ]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    
    try:
        #Tenta acessar atributo 'retweeted_status'
        # Se mensagem não possuir este atributo dá erro
        # e vai para a clausula "except"
        msg.retweeted_status.full_text
    except AttributeError:  
        #Entra aqui sempre que tweet não for retweet
        msgs.append(msg.full_text.lower())

    
    i += 1
    
    temporario=list(set(list(msgs)))
    if len(temporario) >= n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

In [ ]:
msgs=list(set(list(msgs)))

Salvando os dados em uma planilha Excel:

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

### Etapa 2.5 - Transformando o excel em um arquivo .txt

Depois de classificar o excel a mão, transformei em aqruivos txt, para utilizar no classificador

In [ ]:
Valorant = pd.read_excel("Valorant.xlsx")

In [ ]:
relevante = Valorant[Valorant.Classe == 1]
irrelevante = Valorant[Valorant.Classe == 0]

In [ ]:
s_relevante = pd.Series(list(relevante.Treinamento))
s_irrelevante = pd.Series(list(irrelevante.Treinamento))

In [ ]:
relevante_ = []
for tweets in series_relevante:
    relevante_.append(tweets)

relevante_texto = ', '.join(relevante_lista)

with open('relevante.txt','w', encoding="utf8") as arquivo:
    arquivo.write(relevante_texto)

In [ ]:
irrelevante_ = []
for tweets in series_irrelevante:
    irrelevante_.append(tweets)
    
irrelevante_texto = ', '.join(irrelevante_lista)

with open('irrelevante.txt','w', encoding="utf8") as arquivo:
    arquivo.write(irrelevante_texto)